In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("btc_price_prediction") \
    .config(
        "spark.jars.packages",
        "org.postgresql:postgresql:42.7.3"
    ) \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")



your 131072x1 screen size is bogus. expect trouble
26/01/22 23:15:05 WARN Utils: Your hostname, DESKTOP-9V99V1S resolves to a loopback address: 127.0.1.1; using 172.20.61.196 instead (on interface eth0)
26/01/22 23:15:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/Users/khadija/Desktop/BTC-Prediction/quantai-btc-price-prediction-platform/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/khadija/.ivy2/cache
The jars for the packages stored in: /home/khadija/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7621fe7f-4b61-48b5-a1a9-13eb2cea4151;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 322ms :: artifacts dl 11ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------------

In [2]:
#spark.stop()

## 1. Data Injection (collecte data)

In [3]:
import requests
import os
from dotenv import load_dotenv
load_dotenv() 
def data_collection():

  SYMBOL = 'BTCUSDT'
  INTERVAL = '1m'   # Intervalle d'une minute
  LIMIT = 3000   
  api='https://api.binance.com/api/v3/klines'
  response = requests.get(api, params={
    "symbol": SYMBOL,
    "interval": INTERVAL,
    "limit": LIMIT
  })
  if response.status_code != 200:
        print(f"Erreur {response.status_code}")
        return None
  api_data=response.json()
  columns = [
        "open_time", "open_price", "high_price", "low_price", "close_price", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ]
  psdf= spark.createDataFrame(api_data, columns)
 
  
  return psdf
  
data = data_collection()
data.show(10)

+-------------+--------------+--------------+--------------+--------------+-----------+-------------+------------------+----------------+---------------------+----------------------+------+
|    open_time|    open_price|    high_price|     low_price|   close_price|     volume|   close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------------+--------------+--------------+--------------+--------------+-----------+-------------+------------------+----------------+---------------------+----------------------+------+
|1769060160000|89928.10000000|89955.58000000|89928.09000000|89955.57000000| 1.96995000|1769060219999|   177180.89043680|             728|           1.10344000|        99238.20307460|     0|
|1769060220000|89955.58000000|89983.50000000|89955.58000000|89975.72000000| 4.10654000|1769060279999|   369449.22611190|            2253|           2.73544000|       246084.67600210|     0|
|1769060280000|89975.72000000|89998.17000000|89975

In [4]:
data.printSchema()

root
 |-- open_time: long (nullable = true)
 |-- open_price: string (nullable = true)
 |-- high_price: string (nullable = true)
 |-- low_price: string (nullable = true)
 |-- close_price: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: string (nullable = true)
 |-- number_of_trades: long (nullable = true)
 |-- taker_buy_base_volume: string (nullable = true)
 |-- taker_buy_quote_volume: string (nullable = true)
 |-- ignore: string (nullable = true)



In [5]:
print(f"nombre des lignes dana la dataset :",data.count())

nombre des lignes dana la dataset : 1000


In [6]:
from pyspark.sql.functions import col
num_cols= ["open_price", "high_price", "low_price", "close_price", "volume", "quote_asset_volume", "number_of_trades", "taker_buy_base_volume", "taker_buy_quote_volume"]
for feature in num_cols:
      data = data.withColumn(feature, col(feature).cast('double'))

data.printSchema()

root
 |-- open_time: long (nullable = true)
 |-- open_price: double (nullable = true)
 |-- high_price: double (nullable = true)
 |-- low_price: double (nullable = true)
 |-- close_price: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: double (nullable = true)
 |-- number_of_trades: double (nullable = true)
 |-- taker_buy_base_volume: double (nullable = true)
 |-- taker_buy_quote_volume: double (nullable = true)
 |-- ignore: string (nullable = true)



In [7]:
data.describe().show()

+-------+--------------------+------------------+------------------+-----------------+------------------+-----------------+--------------------+------------------+-----------------+---------------------+----------------------+------+
|summary|           open_time|        open_price|        high_price|        low_price|       close_price|           volume|          close_time|quote_asset_volume| number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------+--------------------+------------------+------------------+-----------------+------------------+-----------------+--------------------+------------------+-----------------+---------------------+----------------------+------+
|  count|                1000|              1000|              1000|             1000|              1000|             1000|                1000|              1000|             1000|                 1000|                  1000|  1000|
|   mean|       1.76909013E12| 89689.12637999999| 89716.08627000

1. Convertir les colonnes open_time (ms) et close_time (ms) en secondes (s).

In [8]:


data= data.withColumn("open_time_s", col("open_time") / 1000) \
       .withColumn("close_time_s", col("close_time") / 1000)

data.columns
data.select("open_time_s","open_time").show(5)

+------------+-------------+
| open_time_s|    open_time|
+------------+-------------+
|1.76906016E9|1769060160000|
|1.76906022E9|1769060220000|
|1.76906028E9|1769060280000|
|1.76906034E9|1769060340000|
| 1.7690604E9|1769060400000|
+------------+-------------+
only showing top 5 rows



2. Convertir les colonnes open_time_s et close_time_s  en timestamp.

In [9]:
from pyspark.sql.functions import to_timestamp, col

data = data.withColumn("open_time_ts", to_timestamp(col("open_time_s"))) \
       .withColumn("close_time_ts", to_timestamp(col("close_time_s")))
data.printSchema()



root
 |-- open_time: long (nullable = true)
 |-- open_price: double (nullable = true)
 |-- high_price: double (nullable = true)
 |-- low_price: double (nullable = true)
 |-- close_price: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: double (nullable = true)
 |-- number_of_trades: double (nullable = true)
 |-- taker_buy_base_volume: double (nullable = true)
 |-- taker_buy_quote_volume: double (nullable = true)
 |-- ignore: string (nullable = true)
 |-- open_time_s: double (nullable = true)
 |-- close_time_s: double (nullable = true)
 |-- open_time_ts: timestamp (nullable = true)
 |-- close_time_ts: timestamp (nullable = true)



### create le target y = close_price(t+10)

In [10]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Creation une fenêtre ordonnée par temps
window = Window.orderBy("open_time_ts")

# Décaler la colonne 'close_price' de 10 lignes (10 minutes)
data_new = data.withColumn("close_t_plus_10", F.lead("close_price", 10).over(window))
data_new.select("close_price","close_t_plus_10","open_time_ts").show()

+-----------+---------------+-------------------+
|close_price|close_t_plus_10|       open_time_ts|
+-----------+---------------+-------------------+
|   89955.57|       89993.23|2026-01-22 06:36:00|
|   89975.72|       90002.23|2026-01-22 06:37:00|
|   89998.17|       90036.18|2026-01-22 06:38:00|
|   89999.99|       90021.01|2026-01-22 06:39:00|
|    89991.0|        90044.0|2026-01-22 06:40:00|
|   90010.62|        90044.0|2026-01-22 06:41:00|
|    90004.5|       90024.42|2026-01-22 06:42:00|
|   90004.51|       89958.84|2026-01-22 06:43:00|
|   90010.62|       89961.55|2026-01-22 06:44:00|
|   90002.16|        89931.2|2026-01-22 06:45:00|
|   89993.23|       89955.15|2026-01-22 06:46:00|
|   90002.23|       89950.67|2026-01-22 06:47:00|
|   90036.18|       89952.69|2026-01-22 06:48:00|
|   90021.01|        90026.3|2026-01-22 06:49:00|
|    90044.0|       90037.67|2026-01-22 06:50:00|
|    90044.0|        90061.0|2026-01-22 06:51:00|
|   90024.42|       90048.51|2026-01-22 06:52:00|


#### create colonne variation du prix (return) : return = close(t−1) / close(t) − close(t−1)

In [11]:
from pyspark.sql import functions as F
data_new = data_new.withColumn("precedent_close",F.lag("close_price",1).over(window))
data_new = data_new.withColumn('returns',(F.col("close_price")- F.col("precedent_close"))/F.col("precedent_close"))
data_new.select("close_price","precedent_close","returns").show(5)

+-----------+---------------+--------------------+
|close_price|precedent_close|             returns|
+-----------+---------------+--------------------+
|   89955.57|           NULL|                NULL|
|   89975.72|       89955.57|2.239994699604946...|
|   89998.17|       89975.72|2.495117571717913...|
|   89999.99|       89998.17|2.022263341584595...|
|    89991.0|       89999.99|-9.98888999877137...|
+-----------+---------------+--------------------+
only showing top 5 rows



#### Moyennes mobiles (5, 10 minutes)

In [12]:
window5t = window.orderBy("open_time_ts").rowsBetween(-5,-1)
window10t = window.orderBy("open_time_ts").rowsBetween(-10,-1)

data_new = data_new.withColumn("MA_5",F.avg("close_price").over(window5t))
data_new = data_new.withColumn("MA_10",F.avg("close_price").over(window10t))
data_new.select("close_price","open_time_ts","MA_5","MA_10").show(10)


+-----------+-------------------+-----------------+-----------------+
|close_price|       open_time_ts|             MA_5|            MA_10|
+-----------+-------------------+-----------------+-----------------+
|   89955.57|2026-01-22 06:36:00|             NULL|             NULL|
|   89975.72|2026-01-22 06:37:00|         89955.57|         89955.57|
|   89998.17|2026-01-22 06:38:00|        89965.645|        89965.645|
|   89999.99|2026-01-22 06:39:00|89976.48666666668|89976.48666666668|
|    89991.0|2026-01-22 06:40:00|       89982.3625|       89982.3625|
|   90010.62|2026-01-22 06:41:00|         89984.09|         89984.09|
|    90004.5|2026-01-22 06:42:00|          89995.1|89988.51166666667|
|   90004.51|2026-01-22 06:43:00|        90000.856|89990.79571428572|
|   90010.62|2026-01-22 06:44:00|        90002.124|89992.51000000001|
|   90002.16|2026-01-22 06:45:00|         90004.25|89994.52222222224|
+-----------+-------------------+-----------------+-----------------+
only showing top 10 

#### Volume et Intensité de Trading (Taker Ratio)

In [13]:
data_new = data_new.withColumn("taker_ratio",F.col("taker_buy_base_volume")/F.col("volume"))
data_new.select("open_time_ts", "close_price", "returns",  "MA_5",  "MA_10",  "taker_ratio", "close_t_plus_10").show(10)



+-------------------+-----------+--------------------+-----------------+-----------------+-------------------+---------------+
|       open_time_ts|close_price|             returns|             MA_5|            MA_10|        taker_ratio|close_t_plus_10|
+-------------------+-----------+--------------------+-----------------+-----------------+-------------------+---------------+
|2026-01-22 06:36:00|   89955.57|                NULL|             NULL|             NULL|  0.560136044062032|       89993.23|
|2026-01-22 06:37:00|   89975.72|2.239994699604946...|         89955.57|         89955.57|  0.666117948443215|       90002.23|
|2026-01-22 06:38:00|   89998.17|2.495117571717913...|        89965.645|        89965.645| 0.8677248770825228|       90036.18|
|2026-01-22 06:39:00|   89999.99|2.022263341584595...|89976.48666666668|89976.48666666668|0.20256888725676486|       90021.01|
|2026-01-22 06:40:00|    89991.0|-9.98888999877137...|       89982.3625|       89982.3625|0.13364608765766933| 

In [14]:
data_new = data_new.dropna(subset=["returns", "MA_5", "MA_10", "taker_ratio","close_t_plus_10"])
# print(data_new.count())
# data_new.printSchema()

In [15]:
silver_columns = [
        "open_time_ts", "open_price", "high_price", "low_price", "close_price", "volume",
        "returns", "MA_5", "MA_10", "taker_ratio", "close_t_plus_10"
    ]
df_silver = data_new.select(*silver_columns)

In [16]:
df_silver.columns

['open_time_ts',
 'open_price',
 'high_price',
 'low_price',
 'close_price',
 'volume',
 'returns',
 'MA_5',
 'MA_10',
 'taker_ratio',
 'close_t_plus_10']

In [35]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler ,StandardScaler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor


def training_evaluation_model(data):
 
    features_cols = ["returns", "MA_5", "MA_10", "taker_ratio"]
    
    target = "close_t_plus_10"

    assembler = VectorAssembler(inputCols=features_cols, outputCol="features",handleInvalid="skip")
    window = Window.orderBy("open_time_ts")
    df = data.withColumn("time_index", F.row_number().over(window))

   
    # Séparation séquentielle
    total_count = df.count()
    split_point = int(0.8 * total_count)


    
    train_data = df.filter(F.col("time_index") <= split_point)
    test_data= df.filter(F.col("time_index") > split_point)
    

    
    
    # Transormations
    
    scaler = StandardScaler(inputCol="features", outputCol="features_scaled", withStd=True, withMean=True)

    models = [
        ("Linear Regression", LinearRegression(featuresCol="features_scaled", labelCol=target)),
        
        ("Random Forest", RandomForestRegressor(featuresCol="features_scaled", labelCol=target, numTrees=100, maxDepth=10, 
                       seed=42)),
        
        ("Gradient Boosting (GBT)", GBTRegressor(featuresCol="features_scaled", labelCol=target, maxIter=100, maxDepth=6, 
              seed=42))
    ]  
    #  Évaluateurs
    evaluator_r2 = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
    evaluator_mae = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mae")
    evaluator_rmse = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    
    best_rmse = float("inf")
    best_model_name = None
    for model_name, model_algo in models:
        print(f"Entraînement du modèle : {model_name}...")
    
        # Créer pipeline
        pipeline = Pipeline(stages=[assembler, scaler, model_algo])

        # Entraîner modèle
        model_fit = pipeline.fit(train_data)
        # Prédiction
        preds = model_fit.transform(test_data)
        
        # Évaluation
        r2 = evaluator_r2.evaluate(preds)
        mae = evaluator_mae.evaluate(preds)
        rmse = evaluator_rmse.evaluate(preds)
        
        print(f"Modèle: {model_name}")

        print(f"R²: {r2:.4f} | MAE: {mae:.2f} | RMSE: {rmse:.2f}")
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_model_fit = model_fit
            best_model_name = model_name


    if best_model_fit :
        save_path  = f"saved_model/{best_model_name}_pipeline"
        model_fit.write().overwrite().save(save_path )
        print(f"\n MEILLEUR MODÈLE : {best_model_name} sauvegardé dans '{save_path}' avec un RMSE de {best_rmse:.2f}")
    
    return best_model_fit

In [36]:
training_evaluation_model(df_silver)


Entraînement du modèle : Linear Regression...


Modèle: Linear Regression
R²: 0.4143 | MAE: 116.59 | RMSE: 144.58
Entraînement du modèle : Random Forest...


Modèle: Random Forest
R²: 0.1109 | MAE: 141.83 | RMSE: 178.13
Entraînement du modèle : Gradient Boosting (GBT)...


Modèle: Gradient Boosting (GBT)
R²: -0.5062 | MAE: 185.50 | RMSE: 231.84



 MEILLEUR MODÈLE : Linear Regression sauvegardé dans 'saved_model/Linear Regression_pipeline' avec un RMSE de 144.58


PipelineModel_2804c02c18a8